##MesoNet4 and MesoInception

First, to download the dataset, the steps in the following link should be followed:
https://github.com/ondyari/FaceForensics

after achieving the downloading script, the following format should be followed to complete the download:


```
python3 download-FaceForensics.py
    <output path>
    -d <dataset type, e.g., Face2Face, original or all>
    -c <compression quality, e.g., c23, raw or c40>
    -t <file type, e.g., videos, masks or models>
```



which is in our case:

`python3 script.py faceforensics -d all -c c40 -t videos --server EU2 `


After cloning https://github.com/HongguLiu/MesoNet-Pytorch, you will access all the required files. For example, the complete architecture implementation of MesoNet and MesoInception:

In [ ]:
import os
import argparse


import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import torchvision

class Meso4(nn.Module):
	def __init__(self, num_classes=2):
		super(Meso4, self).__init__()
		self.num_classes = num_classes
		self.conv1 = nn.Conv2d(3, 8, 3, padding=1, bias=False)
		self.bn1 = nn.BatchNorm2d(8)
		self.relu = nn.ReLU(inplace=True)
		self.leakyrelu = nn.LeakyReLU(0.1)

		self.conv2 = nn.Conv2d(8, 8, 5, padding=2, bias=False)
		self.bn2 = nn.BatchNorm2d(16)
		self.conv3 = nn.Conv2d(8, 16, 5, padding=2, bias=False)
		self.conv4 = nn.Conv2d(16, 16, 5, padding=2, bias=False)
		self.maxpooling1 = nn.MaxPool2d(kernel_size=(2, 2))
		self.maxpooling2 = nn.MaxPool2d(kernel_size=(4, 4))
		self.dropout = nn.Dropout2d(0.5)
		self.fc1 = nn.Linear(16*8*8, 16)
		self.fc2 = nn.Linear(16, num_classes)

	def forward(self, input):
		x = self.conv1(input)
		x = self.relu(x)
		x = self.bn1(x)
		x = self.maxpooling1(x)

		x = self.conv2(x)
		x = self.relu(x)
		x = self.bn1(x)
		x = self.maxpooling1(x)

		x = self.conv3(x)
		x = self.relu(x)
		x = self.bn2(x)
		x = self.maxpooling1(x)

		x = self.conv4(x)
		x = self.relu(x)
		x = self.bn2(x)
		x = self.maxpooling2(x)

		x = x.view(x.size(0), -1)
		x = self.dropout(x)
		x = self.fc1(x)
		x = self.leakyrelu(x)
		x = self.dropout(x)
		x = self.fc2(x)

		return x


class MesoInception4(nn.Module):
	def __init__(self, num_classes=2):
		super(MesoInception4, self).__init__()
		self.num_classes = num_classes
		self.Incption1_conv1 = nn.Conv2d(3, 1, 1, padding=0, bias=False)
		self.Incption1_conv2_1 = nn.Conv2d(3, 4, 1, padding=0, bias=False)
		self.Incption1_conv2_2 = nn.Conv2d(4, 4, 3, padding=1, bias=False)
		self.Incption1_conv3_1 = nn.Conv2d(3, 4, 1, padding=0, bias=False)
		self.Incption1_conv3_2 = nn.Conv2d(4, 4, 3, padding=2, dilation=2, bias=False)
		self.Incption1_conv4_1 = nn.Conv2d(3, 2, 1, padding=0, bias=False)
		self.Incption1_conv4_2 = nn.Conv2d(2, 2, 3, padding=3, dilation=3, bias=False)
		self.Incption1_bn = nn.BatchNorm2d(11)

		self.Incption2_conv1 = nn.Conv2d(11, 2, 1, padding=0, bias=False)
		self.Incption2_conv2_1 = nn.Conv2d(11, 4, 1, padding=0, bias=False)
		self.Incption2_conv2_2 = nn.Conv2d(4, 4, 3, padding=1, bias=False)
		self.Incption2_conv3_1 = nn.Conv2d(11, 4, 1, padding=0, bias=False)
		self.Incption2_conv3_2 = nn.Conv2d(4, 4, 3, padding=2, dilation=2, bias=False)
		self.Incption2_conv4_1 = nn.Conv2d(11, 2, 1, padding=0, bias=False)
		self.Incption2_conv4_2 = nn.Conv2d(2, 2, 3, padding=3, dilation=3, bias=False)
		self.Incption2_bn = nn.BatchNorm2d(12)

		self.conv1 = nn.Conv2d(12, 16, 5, padding=2, bias=False)
		self.relu = nn.ReLU(inplace=True)
		self.leakyrelu = nn.LeakyReLU(0.1)
		self.bn1 = nn.BatchNorm2d(16)
		self.maxpooling1 = nn.MaxPool2d(kernel_size=(2, 2))

		self.conv2 = nn.Conv2d(16, 16, 5, padding=2, bias=False)
		self.maxpooling2 = nn.MaxPool2d(kernel_size=(4, 4))

		self.dropout = nn.Dropout2d(0.5)
		self.fc1 = nn.Linear(16*8*8, 16)
		self.fc2 = nn.Linear(16, num_classes)

	def InceptionLayer1(self, input):
		x1 = self.Incption1_conv1(input)
		x2 = self.Incption1_conv2_1(input)
		x2 = self.Incption1_conv2_2(x2)
		x3 = self.Incption1_conv3_1(input)
		x3 = self.Incption1_conv3_2(x3)
		x4 = self.Incption1_conv4_1(input)
		x4 = self.Incption1_conv4_2(x4)
		y = torch.cat((x1, x2, x3, x4), 1)
		y = self.Incption1_bn(y)
		y = self.maxpooling1(y)

		return y

	def InceptionLayer2(self, input):
		x1 = self.Incption2_conv1(input)
		x2 = self.Incption2_conv2_1(input)
		x2 = self.Incption2_conv2_2(x2)
		x3 = self.Incption2_conv3_1(input)
		x3 = self.Incption2_conv3_2(x3)
		x4 = self.Incption2_conv4_1(input)
		x4 = self.Incption2_conv4_2(x4)
		y = torch.cat((x1, x2, x3, x4), 1)
		y = self.Incption2_bn(y)
		y = self.maxpooling1(y)

		return y

	def forward(self, input):
		x = self.InceptionLayer1(input)
		x = self.InceptionLayer2(x)

		x = self.conv1(x)
		x = self.relu(x)
		x = self.bn1(x)
		x = self.maxpooling1(x)

		x = self.conv2(x)
		x = self.relu(x)
		x = self.bn1(x)
		x = self.maxpooling2(x)

		x = x.view(x.size(0), -1)
		x = self.dropout(x)
		x = self.fc1(x)
		x = self.leakyrelu(x)
		x = self.dropout(x)
		x = self.fc2(x)

		return x


Install the python packages with:

In [ ]:
!python3 -m pip install -r requirements.txt

To train the MesoNet4

In [ ]:
!python3 train_Meso.py -n 'Mesonet' -tp './data/train' -vp './data/val' -bz 64 -e 100 -mn 'meso4.pkl'

To train the MesoInceptionNet4

In [ ]:
!python3 train_MesoInception.py -n 'MesoInception' -tp './data/train' -vp './data/val' -bz 64 -e 100 -mn 'mesoinception.pkl'

To test the trained model

In [ ]:
!python3 test.py -bz 64 -tp './data/test' -mp './Mesonet/best.pkl'

##Xception

After cloning https://github.com/i3p9/deepfake-detection-with-xception:

Put FF++ dataset under the ./dataset.

To do that, move the FaceForensics++ data to the appropriate paths:

In [ ]:
!mv faceforensics/original_sequences/youtube/c23/videos/* /home/paperspace/deepfake-detection-with-xception/dataset/real
!mv faceforensics/manipulated_sequences/Deepfakes/c23/videos* /home/paperspace/deepfake-detection-with-xception/dataset/fake

Update the train_dateset.py file as the following and rename it to train_dataset.py.

In [ ]:
import os
import math
import argparse
import matplotlib
import imghdr
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt
from keras.applications.xception import Xception, preprocess_input
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.losses import categorical_crossentropy
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

# Suppress TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

matplotlib.use('Agg')
current_directory = os.path.dirname(os.path.abspath(__file__))
parser = argparse.ArgumentParser()
parser.add_argument('dataset_root')
parser.add_argument('classes')
parser.add_argument('result_root')
parser.add_argument('--epochs_pre', type=int, default=10)
parser.add_argument('--epochs_fine', type=int, default=30)
parser.add_argument('--batch_size_pre', type=int, default=32)
parser.add_argument('--batch_size_fine', type=int, default=16)
parser.add_argument('--lr_pre', type=float, default=5e-3)
parser.add_argument('--lr_fine', type=float, default=5e-4)
parser.add_argument('--snapshot_period_pre', type=int, default=1)
parser.add_argument('--snapshot_period_fine', type=int, default=1)
parser.add_argument('--split', type=float, default=0.8)


def generate_from_paths_and_labels(input_paths, labels, batch_size, input_size=(299, 299)):
    num_samples = len(input_paths)
    while 1:
        perm = np.random.permutation(num_samples)
        input_paths = input_paths[perm]
        labels = labels[perm]
        for i in range(0, num_samples, batch_size):
            inputs = list(map(
                lambda x: image.load_img(x, target_size=input_size),
                input_paths[i:i+batch_size]
            ))
            inputs = np.array(list(map(
                lambda x: image.img_to_array(x),
                inputs
            )))
            inputs = preprocess_input(inputs)
            yield (inputs, labels[i:i+batch_size])


def main(args):
    print("Starting the script...")

    epochs = args.epochs_pre + args.epochs_fine
    args.dataset_root = os.path.expanduser(args.dataset_root)
    args.result_root = os.path.expanduser(args.result_root)
    args.classes = os.path.expanduser(args.classes)

    # load class names
    print("Loading class names from:", args.classes)
    with open(args.classes, 'r') as f:
        classes = f.readlines()
        classes = list(map(lambda x: x.strip(), classes))
    num_classes = len(classes)

    # make input_paths and labels
    input_paths, labels = [], []
    for class_name in os.listdir(args.dataset_root):
        class_root = os.path.join(args.dataset_root, class_name)
        if not os.path.isdir(class_root):
            continue
        print("Processing class:", class_name)
        class_id = classes.index(class_name)
        for path in os.listdir(class_root):
            path = os.path.join(class_root, path)
            if imghdr.what(path) is None:
                # this is not an image file
                continue
            input_paths.append(path)
            labels.append(class_id)

    print("Converting labels to one-hot-vector format...")
    labels = to_categorical(labels, num_classes=num_classes)

    print("Converting input paths to numpy array...")
    input_paths = np.array(input_paths)

    # shuffle dataset
    print("Shuffling the dataset...")
    perm = np.random.permutation(len(input_paths))
    labels = labels[perm]
    input_paths = input_paths[perm]

    # split dataset for training and validation
    border = int(len(input_paths) * args.split)
    train_labels = labels[:border]
    val_labels = labels[border:]
    train_input_paths = input_paths[:border]
    val_input_paths = input_paths[border:]
    print("Training on %d images and labels" % (len(train_input_paths)))
    print("Validation on %d images and labels" % (len(val_input_paths)))

    if os.path.exists(args.result_root) is False:
        os.makedirs(args.result_root)

    print("Building the model...")
    # Build a custom Xception from pre-trained Xception model
    base_model = Xception(
        include_top=False,
        weights='imagenet',
        input_shape=(299, 299, 3))

    # create a custom top classifier
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.inputs, outputs=predictions)

    # Train only the top classifier
    # freeze the body layers
    for layer in base_model.layers:
        layer.trainable = False

    # compile model
    print("Compiling the model for pre-training...")
    model.compile(
        loss=categorical_crossentropy,
        optimizer=Adam(learning_rate=args.lr_pre),
        metrics=['accuracy']
    )

    # train
    print("Starting pre-training...")
    hist_pre = model.fit(
        generate_from_paths_and_labels(
            input_paths=train_input_paths,
            labels=train_labels,
            batch_size=args.batch_size_pre
        ),
        steps_per_epoch=math.ceil(
            len(train_input_paths) / args.batch_size_pre),
        epochs=args.epochs_pre,
        validation_data=generate_from_paths_and_labels(
            input_paths=val_input_paths,
            labels=val_labels,
            batch_size=args.batch_size_pre
        ),
        validation_steps=math.ceil(
            len(val_input_paths) / args.batch_size_pre),
        verbose=1,
        callbacks=[
            ModelCheckpoint(
                filepath=os.path.join(
                    args.result_root,
                    'model_pre_ep{epoch}_acc{accuracy:.3f}.keras'),
                save_freq=args.snapshot_period_pre * len(train_input_paths) // args.batch_size_pre,
            ),
        ],
    )
    model.save(os.path.join(args.result_root, 'model_pre_final.keras'))

    # Train the whole model
    print("Starting fine-tuning...")
    for layer in model.layers:
        layer.trainable = True  # all layers are set as trainable

    # recompile
    model.compile(
        optimizer=Adam(learning_rate=args.lr_fine),
        loss=categorical_crossentropy,
        metrics=['accuracy'])

    # train
    hist_fine = model.fit(
        generate_from_paths_and_labels(
            input_paths=train_input_paths,
            labels=train_labels,
            batch_size=args.batch_size_fine
        ),
        steps_per_epoch=math.ceil(
            len(train_input_paths) / args.batch_size_fine),
        epochs=args.epochs_fine,
        validation_data=generate_from_paths_and_labels(
            input_paths=val_input_paths,
            labels=val_labels,
            batch_size=args.batch_size_fine
        ),
        validation_steps=math.ceil(
            len(val_input_paths) / args.batch_size_fine),
        verbose=1,
        callbacks=[
            ModelCheckpoint(
                filepath=os.path.join(
                    args.result_root,
                    'model_fine_ep{epoch}_acc{accuracy:.3f}.keras'),
                save_freq=args.snapshot_period_fine * len(train_input_paths) // args.batch_size_fine,
            ),
        ],
    )
    model.save(os.path.join(args.result_root, 'model_fine_final.keras'))

    # Create result graphs
    print("Generating result graphs...")
    acc = hist_pre.history['accuracy']
    val_acc = hist_pre.history['val_accuracy']
    loss = hist_pre.history['loss']
    val_loss = hist_pre.history['val_loss']
    acc.extend(hist_fine.history['accuracy'])
    val_acc.extend(hist_fine.history['val_accuracy'])
    loss.extend(hist_fine.history['loss'])
    val_loss.extend(hist_fine.history['val_loss'])

    # save graph image
    plt.plot(range(epochs), acc, marker='.', label='accuracy')
    plt.plot(range(epochs), val_acc, marker='.', label='val_accuracy')
    plt.legend(loc='best')
    plt.grid()
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.savefig(os.path.join(args.result_root, 'accuracy.png'))
    plt.clf()

    plt.plot(range(epochs), loss, marker='.', label='loss')
    plt.plot(range(epochs), val_loss, marker='.', label='val_loss')
    plt.legend(loc='best')
    plt.grid()
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.savefig(os.path.join(args.result_root, 'loss.png'))
    plt.clf()

    # save plot data
    print("Saving plot data...")
    plot = {
        'accuracy': acc,
        'val_accuracy': val_acc,
        'loss': loss,
        'val_loss': val_loss,
    }
    with open(os.path.join(args.result_root, 'plot.dump'), 'wb') as f:
        pkl.dump(plot, f)

    print("Script completed successfully.")


if __name__ == '__main__':
    args = parser.parse_args()
    main(args)

Then, run the training script:

In [ ]:
!python3 train_dataset.py dataset dataset/classes.txt deepfake-detection-with-xception/results

##Face X-Ray

After cloning https://github.com/AlgoHunt/Face-Xray ,
just run bi_online_generation.py file to generate the X-Rays

In [ ]:
!python3 bi_online_generation.py

##Implicit Identity Leakage: The Stumbling Block to Improving Deepfake Detection Generalization

After cloning https://github.com/megvii-research/CADDM:

Put FF++ dataset under the ./data.

Download the landmark detector from https://github.com/codeniko/shape_predictor_81_face_landmarks and put it in the folder ./lib.

Run the code to extract frames from FF++ videos and save them under the ./train_images or ./test_images based on the division in the original dataset.

In [ ]:
!python3 lib/extract_frames_ldm_ff++.py

Evaluation:

In [ ]:
!python3 test.py --cfg ./configs/caddm_test.cfg

Training:

In [ ]:
!python3 train.py --cfg ./configs/caddm_train.cfg

###PS: For Cross-Dataset Testing of Each Method

Download the Celeb-DF dataset using the steps in the link: https://github.com/yuezunli/celeb-deepfakeforensics